<a href="https://colab.research.google.com/github/e-corwin/ImageClassification/blob/main/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building the Neural Net

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Getting the Device for Training

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


Defining the Class

In [3]:
class NeuralNetwork(nn.Module): # subclass is the nn.Module
    def __init__(self): # initialize the neural network layers
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device) # Creating an instance of NeuralNetwork and moving it to the device
print(model) # printing the structure

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Calling the Model on the Input

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}") # print the prediction probabilities

Predicted class: tensor([0])


## The Model Layers
### Examples

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten() # converts each 2D 28x28 image into a contiguous array of 784 pixel values
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20) # applies a linear transformation on the input using its stored weights and biases
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1) # creates non-linearity within model
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1467,  0.1080, -0.2426,  0.2484, -0.6917,  0.3355, -0.0588, -0.1320,
          0.0372,  0.2928,  0.6372,  0.1677,  0.0536, -0.1630, -0.3007, -0.2413,
          0.4288, -0.2520, -0.3235, -0.0867],
        [-0.1500,  0.1715, -0.4086,  0.3528, -0.6360,  0.5586,  0.0400,  0.1108,
          0.1221,  0.4953,  0.4254,  0.1632, -0.0239, -0.2803,  0.0045,  0.0515,
          0.3115, -0.5577,  0.0388, -0.2585],
        [-0.0852,  0.2278, -0.2596,  0.5504, -0.5916,  0.2751, -0.1477, -0.0073,
          0.2557,  0.3636,  0.4612,  0.1173,  0.3703, -0.1943, -0.2020,  0.2072,
          0.6389, -0.5297, -0.1076, -0.5045]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1080, 0.0000, 0.2484, 0.0000, 0.3355, 0.0000, 0.0000, 0.0372,
         0.2928, 0.6372, 0.1677, 0.0536, 0.0000, 0.0000, 0.0000, 0.4288, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.1715, 0.0000, 0.3528, 0.0000, 0.5586, 0.0400, 0.1108, 0.1221,
         0.4953, 0.4254, 0.1632, 0.0000, 0.0000, 0.00

In [10]:
seq_modules = nn.Sequential( # creates an ordered container of modules
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits) # returns logits which are passed to the nn.Softmax module
# Logits: raw values [-infty, infty]

## The Model Parameters

In [12]:
print(f"Model structure: {model}\n\n")

# Iterating over parameters and printing size and preview of its values
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-3.3332e-02, -1.4135e-06, -1.6943e-02,  ..., -7.9878e-03,
          9.0768e-04,  2.9133e-02],
        [ 8.1029e-04, -1.1520e-02, -3.0498e-02,  ..., -2.3339e-05,
          6.7149e-03, -3.2393e-02]], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0108,  0.0275], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0342,  0.0347,  0.0430,  ...,  0.0422, -0.0268, -0.0008],
        [-0.0423, -0.0238, -0.0026,  ...,  0.0190, -0.0240,  0.0189]],
       gra

# Automatic Differentiation with Torch.Autograd


In [13]:
# torch.autograd supports automatic computation of gradient for any computational graph
# gradients are then used in back propagation

In [14]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [15]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7fa873960c50>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7fa873960b10>


## Computing Gradients

In [16]:
loss.backward() # compute the derivatives of loss/w and loss/b
print(w.grad)
print(b.grad)

tensor([[0.3069, 0.2634, 0.1996],
        [0.3069, 0.2634, 0.1996],
        [0.3069, 0.2634, 0.1996],
        [0.3069, 0.2634, 0.1996],
        [0.3069, 0.2634, 0.1996]])
tensor([0.3069, 0.2634, 0.1996])


## Disabling Gradient Tracking

In [17]:
# used to mark some parameters as frozen parameters or to speed up computations when only doing a forward pass
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


# Optimizing Model Parameters

In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## Setting the Hyperparamters

In [19]:
learning_rate = 1e-3 # how much to update models parameters at each batch/epoch
batch_size = 64 # the number of data samples propagated through the network before the parameters are updated
epochs = 5 # the number times to iterate over the dataset

## Optimization Loop

In [20]:
# Initialize the loss function
# measures the degree of dissimilarity of obtained result to the target value
# want to minimize this
loss_fn = nn.CrossEntropyLoss()

## Optimizer

In [21]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Defining the train_loop and test_loop

In [22]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301588  [    0/60000]
loss: 2.285755  [ 6400/60000]
loss: 2.261456  [12800/60000]
loss: 2.253461  [19200/60000]
loss: 2.235631  [25600/60000]
loss: 2.212582  [32000/60000]
loss: 2.209984  [38400/60000]
loss: 2.176879  [44800/60000]
loss: 2.179763  [51200/60000]
loss: 2.137372  [57600/60000]
Test Error: 
 Accuracy: 53.1%, Avg loss: 2.132962 

Epoch 2
-------------------------------
loss: 2.149368  [    0/60000]
loss: 2.136535  [ 6400/60000]
loss: 2.074043  [12800/60000]
loss: 2.083285  [19200/60000]
loss: 2.028986  [25600/60000]
loss: 1.976877  [32000/60000]
loss: 1.987412  [38400/60000]
loss: 1.911506  [44800/60000]
loss: 1.924690  [51200/60000]
loss: 1.833369  [57600/60000]
Test Error: 
 Accuracy: 55.8%, Avg loss: 1.838778 

Epoch 3
-------------------------------
loss: 1.885584  [    0/60000]
loss: 1.845839  [ 6400/60000]
loss: 1.730972  [12800/60000]
loss: 1.756725  [19200/60000]
loss: 1.649606  [25600/60000]
loss: 1.617199  [32000/600

# Saving and Loading the Model

In [24]:
import torch
import torchvision.models as models

## Saving and Loading the Model Weights

In [25]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Saving and Loading Models with Shapes

In [26]:
torch.save(model, 'model.pth') # saving
model = torch.load('model.pth') # loading